In [15]:
# === Cell 1: 1D Burgers' equation simulation and save to CSV ===
import numpy as np
import pandas as pd

# Parameters
nx = 100
nt = 200
Lx = 2.0 * np.pi
dx = Lx / nx
dt = 0.001
nu = 0.01  # viscosity

x = np.linspace(0, Lx, nx)
t = np.arange(0, nt*dt, dt)

# Initialize: sine wave
u = np.zeros((nx, nt))
u[:,0] = np.sin(x)

# Time stepping (explicit scheme)
for k in range(nt-1):
    u_x = (np.roll(u[:,k], -1) - np.roll(u[:,k], 1)) / (2*dx)
    u_xx = (np.roll(u[:,k], -1) - 2*u[:,k] + np.roll(u[:,k], 1)) / (dx**2)
    u[:,k+1] = u[:,k] + dt * (-u[:,k]*u_x + nu*u_xx)

# Save to CSV
X, T = np.meshgrid(x, t, indexing="ij")
df = pd.DataFrame({
    "x": X.ravel(),
    "t": T.ravel(),
    "u": u.ravel()
})
df.to_csv("burgers_rawdata.csv", index=False)
print("Saved burgers_rawdata.csv with shape:", df.shape)


Saved burgers_rawdata.csv with shape: (20000, 3)


In [16]:
# === Cell 2: Load Data ===
df = pd.read_csv("burgers_rawdata.csv")
x = np.unique(df["x"].values)
t = np.unique(df["t"].values)
u = df["u"].values.reshape(len(x), len(t))
print("u.shape =", u.shape)


u.shape = (100, 200)


In [17]:
# === Cell 3: Higher-order derivatives ===
nx, nt = len(x), len(t)
dx = x[1]-x[0]
dt = t[1]-t[0]

u_t   = np.zeros_like(u)
u_2t  = np.zeros_like(u)
u_x   = np.zeros_like(u)
u_2x  = np.zeros_like(u)
u_3x  = np.zeros_like(u)

for k in range(1, nt-1):
    for i in range(2, nx-2):
        u_t[i,k]  = (u[i,k+1] - u[i,k-1]) / (2*dt)
        u_2t[i,k] = (u[i,k+1] - 2*u[i,k] + u[i,k-1]) / (dt**2)

        u_x[i,k]  = (u[i+1,k] - u[i-1,k]) / (2*dx)
        u_2x[i,k] = (u[i+1,k] - 2*u[i,k] + u[i-1,k]) / (dx**2)
        u_3x[i,k] = (u[i+2,k] - 2*u[i+1,k] + 2*u[i-1,k] - u[i-2,k]) / (2*dx**3)

print("Derivatives computed: u_t, u_2t, u_x, u_2x, u_3x")


Derivatives computed: u_t, u_2t, u_x, u_2x, u_3x


In [18]:
# === Cell 4: Candidate Library ===
U    = u.ravel()
Ut   = u_t.ravel()
U2t  = u_2t.ravel()
Ux   = u_x.ravel()
U2x  = u_2x.ravel()
U3x  = u_3x.ravel()

Theta = np.column_stack([
    U,        # u
    Ux,       # u_x
    U2x,      # u_xx
    U3x,      # u_xxx
    U*Ux      # nonlinear u*u_x
])

# Python-safe names for geppy
feature_names = ["u", "ux", "uxx", "uxxx", "u_ux"]

print("Theta shape:", Theta.shape)


Theta shape: (20000, 5)


In [19]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1e-6, fit_intercept=False)
ridge.fit(Theta, Ut)
coeffs = ridge.coef_

print("Ridge Regression PDE:")
print("u_t =")
for name, c in zip(feature_names, coeffs):
    if abs(c) > 1e-8:
        print(f"  {c:.5f} * {name}")


Ridge Regression PDE:
u_t =
  0.00018 * u
  0.01026 * uxx
  -1.01012 * u_ux


In [20]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=1e-4, fit_intercept=False, max_iter=20000)
lasso.fit(Theta, Ut)
coeffs_lasso = lasso.coef_

print("Lasso Regression PDE:")
print("u_t =")
for name, c in zip(feature_names, coeffs_lasso):
    if abs(c) > 1e-8:
        print(f"  {c:.5f} * {name}")


Lasso Regression PDE:
u_t =
  -0.00667 * u
  0.00329 * uxx
  -1.00712 * u_ux


In [30]:
# === Cell 7: GEP with geppy (final imports) ===
# !pip install geppy deap

import geppy as gep
import operator
import numpy as np
from geppy import mutation, crossover, transposition
from deap import base, tools, algorithms

# Primitive set
pset = gep.PrimitiveSet('PDE', input_names=feature_names)
pset.add_function(operator.add, 2)
pset.add_function(operator.sub, 2)
pset.add_function(operator.mul, 2)
pset.add_function(operator.truediv, 2)

# Ephemeral constants
pset.add_ephemeral_terminal("rand", lambda: np.random.uniform(-2, 2))

# Gene generator
def gene_gen():
    return gep.Gene(pset=pset, head_length=5)

# Chromosome: 3 genes linked by addition
ind = gep.Chromosome(gene_gen, n_genes=3, linker=operator.add)

# Fitness
def fitness(individual):
    func = gep.compile(individual, pset)
    try:
        y_pred = np.array([func(*row) for row in Theta])
        return -np.mean((y_pred - Ut)**2),
    except Exception:
        return -1e6,

# DEAP toolbox
toolbox = base.Toolbox()
toolbox.register("evaluate", fitness)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", crossover.one_point)
toolbox.register("mutate", mutation.uniform_point)
toolbox.register("is_transposition", transposition.is_transposition)
toolbox.register("ris_transposition", transposition.ris_transposition)
toolbox.register("gene_transposition", transposition.gene_transposition)

# Population
pop = gep.GepPopulation(30, ind)

# Evolution
algorithms.eaSimple(pop, toolbox, cxpb=0.4, mutpb=0.1, ngen=20, verbose=True)

# Best individual
best = tools.selBest(pop, 1)[0]
print("Best GEP expression:")
print(gep.compile(best, pset))


ImportError: cannot import name 'mutation' from 'geppy' (c:\Users\adil\anaconda3\envs\ham\Lib\site-packages\geppy\__init__.py)